# Scraping with Pandas

In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

### NASA Mars News
Scrape the Mars News Site (url) to collect the latest News Titles and Paragraph text. 
Save these to variables to use them later.

Initialize PyMongo to work with MongoDBs

In [ ]:
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

Define database and collection

In [ ]:
# db = client.commerce_db
# collection = db.items

Set up URL
Retrieve page with requests module
Create BeautifulSoup object; parse with 'lxml'

In [2]:
# url = 'https://redplanetscience.com/'
# response = requests.get(url)
# soup = BeautifulSoup(response.text, 'lxml')

Examine the results, determine element that contains the title and paragraph. 

In [ ]:
# results = soup.find_all('div', class_='caption')

# # Loop through returned results
# for result in results:
#     # Error handling
#     try:
#         # Identify and return title of listing
#         title = result.find('a', class_='title').text
#         # Identify and return price of listing
#         price = result.find('h4', class_='price').text
#         # Identify and return link to listing
#         link = result.a['href']

#         # Run only if title, price, and link are available
#         if (title and price and link):
#             # Print results
#             print('-------------')
#             print(title)
#             print(price)
#             print(link)

#             # Dictionary to be inserted as a MongoDB document
#             post = {
#                 'title': title,
#                 'price': price,
#                 'url': link
#             }

#             collection.insert_one(post)

#     except Exception as e:
#         print(e)

In [ ]:
# Display items in MongoDB collection
# listings = db.items.find()

# for listing in listings:
#     print(listing)

In [ ]:
# browser.quit()

### JPL Mars Space Images - Featured Image
Visit the Featured Space Image site (image_url)to collect images.
Use splitner to navigate to the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
Make sure to find the image url to the full size .jpg image.
Make sure to save a complete url string for this image: for example:
featured_image_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'

#### set up splinter

In [ ]:
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# image_url = 'https://spaceimages-mars.com/'
# browser.visut.(image_url)

In [ ]:
# for x in range(1, 6):

#     html = browser.html
#     soup = BeautifulSoup(html, 'html.parser')

#     quotes = soup.find_all('span', class_='text')

#     for quote in quotes:
#         print('page:', x, '-------------')
#         print(quote.text)

#     browser.links.find_by_partial_text('Next').click()

In [ ]:
# broswer.quit()

### Mars Facts
Visit the Mars Facts webpage (facts_url) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
Use Pandas to convert the data to a HTML table string.

In [ ]:
# facts_url = 'https://galaxyfacts-mars.com/'

In [ ]:
# tables = pd.read_html(url)
# tables

In [ ]:
#return is a list of dataframes for any tabular data that Pandas found
# type(tables)

In [ ]:
#slice off dataframes that we want using normal indexing
# df = tables[0]
# df.head()

In [ ]:
#drop single header rows
# df.columns = df.columns.get_level_values(0)
# df = df.loc[df.Ref.str.startswith("[")]
# df.head()

In [ ]:
# Split the column values into separate columsn as needed
# columnsplit = df['City'].str.split(", ", expand=True)
# df = df.assign(City=columnsplit[0],State=columnsplit[1])
# df.head()

# #drop a column
# df=df.drop(['Ref'],axis=1)

# #Rest an index
# df = df.reset_index(drop=True)

In [ ]:
# browser.quit()

### Mars Hemispheres
Visit the astrogeology site (astro_url) to obtain high resolution images for each of Mars's hemispheres.

- Click each of the links to the hemispheres in order to find the image url to the full resolution image.
- Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
- Use a Python dictionary to store the data using the keys img_url and title.
- Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [ ]:
astro_url = 'https://marshemispheres.com/'

## DataFrames as HTML

#### Pandas also had a `to_html` method that we can use to generate HTML tables from DataFrames.

In [ ]:
html_table = df.to_html()
html_table

#### You may have to strip unwanted newlines to clean up the table.

In [ ]:
html_table.replace('\n', '')

You can also save the table directly to a file.

In [ ]:
df.to_html('table.html')

In [ ]:
# OSX Users can run this to open the file in a browser, 
# or you can manually find the file and open it in the browser
!open table.html